In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
## Read Data
batch1 = pd.read_parquet("C:/Users/k_vsl/Documents/Erdos/IceCubeData/batch_1.parquet/batch_1.parquet")
directions = pd.read_parquet("C:/Users/k_vsl/Documents/Erdos/IceCubeData/train_meta.parquet/train_meta.parquet")
sensors = pd.read_csv("C:/Users/k_vsl/Documents/Erdos/IceCubeData/sensor_geometry.csv")

In [6]:
batch1.head()

,sensor_id,time,charge,auxiliary
event_id,,,,
24,3918,5928,1.325,True
24,4157,6115,1.175,True
24,3520,6492,0.925,True
24,5041,6665,0.225,True
24,2948,8054,1.575,True


In [84]:
# Modelling approach as detailed in Ice Cube Papers
# r = r_0 + vt
# Get a formula for v as 
# v = (<r_it_i> - <r_i><t_i>)/(<t_i^2> - <t_i>^2)

def get_velocity(event_id, 
              aux_incl, 
              batch,
              geometry):
    
    # Isolate the detections we want to use
    detections = (batch.loc[event_id])
    if aux_incl == False: 
        detections = detections[detections["auxiliary"] == False]
    
    time0 = detections["time"].iloc[0]
    detections["t"] = detections["time"] - time0
        
    # Get info for each detection
    detections["x"] = [geometry.iloc[s].x for s in detections["sensor_id"]]
    detections["y"] = [geometry.iloc[s].y for s in detections["sensor_id"]]
    detections["z"] = [geometry.iloc[s].z for s in detections["sensor_id"]]
    detections["xt"] = detections["x"] * detections["t"]
    detections["yt"] = detections["y"] * detections["t"]
    detections["zt"] = detections["z"] * detections["t"]
    detections["t2"] = detections["t"]^2
    
    # Preliminary Averages
    means = detections.mean(axis=0)
    
    # Compute Velocity
    denom = (means.t2 - np.square(means.t))
    vx = (means.xt - means.x*means.t)/denom
    vy = (means.yt - means.y*means.t)/denom
    vz = (means.zt - means.z*means.t)/denom
    
    return (vx,vy,vz)
    
    
    

In [85]:
m = get_velocity(24, False, batch1, sensors)

In [238]:
# Function giving azimuth and zenith based on velocity
def get_angle(v): 
    mod2 = np.square(v[0]) + np.square(v[1]) + np.square(v[2])
    mod = np.sqrt(mod2)
    x = v[0] / mod
    y = v[1] / mod
    z = v[2] / mod
    ze = np.arccos(z)
    az = np.arctan2(y,x)
    return (az,ze)

In [239]:
d = get_angle(m)
d

(1.6050174265309547, 2.6687351836307927)

In [240]:
# Function to compute Mean Angular error
def get_mae(az_true, zen_true, az_pred, zen_pred): 
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)
    
    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)
    
    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
    # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod =  np.clip(scalar_prod, -1, 1)
    
    # convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))

In [206]:
az_true, zen_true = directions.loc[0, ["azimuth", "zenith"]]
print(az_true, zen_true)

5.029554633390068 2.0874975005610428


In [101]:
mae(az_true, zen_true, d[0], d[1])

2.545789300015317

In [217]:
def find_event_error(event_id, aux_incl, batch, geometry, directions):
    # Get velocity for event
    print("Event ID: " + str(event_id))
    v = get_velocity(event_id, aux_incl, batch, geometry)
    (az_pred, ze_pred) = get_angle(v)
    print("The azimuth predicted is " + str(az_pred) + " and the zenith predicted is " + str(ze_pred))
    df = directions[directions.event_id == event_id]
    ze = df.iloc[0].zenith
    az = df.iloc[0].azimuth
    print("The azimuth is " + str(az) + " and the zenith is " + str(ze))
    mae = get_mae(az, ze, az_pred, ze_pred)
    print("The MAE is " + str(mae))
    
    
    return mae

In [242]:
# Package functions together
def find_error(aux_incl, batch, geometry, directions): 
    event_ids = list(set(batch.index))
    n = len(event_ids)
    print(n)
    data = pd.DataFrame(event_ids, columns = ["event_id"])
    data["mae"] = [find_event_error(s, aux_incl, batch, geometry, directions) for s in event_ids]
    
    return data

In [241]:
find_event_error(2100159, False, batch1, sensors, directions)

Event ID: 2100159
The azimuth predicted is 2.5860086747198454 and the zenith predicted is 0.4287159232589412
The azimuth is 4.196398388699694 and the zenith is 0.8572350143887308
The MAE is 0.9485583187503158


0.9485583187503158

In [233]:
event24 = batch1.loc[24]

In [234]:
event24.head()

,sensor_id,time,charge,auxiliary
event_id,,,,
24,3918,5928,1.325,True
24,4157,6115,1.175,True
24,3520,6492,0.925,True
24,5041,6665,0.225,True
24,2948,8054,1.575,True


In [235]:
find_error(False, event24, sensors, directions)

1
Event ID: 24
The azimuth predicted is 1.6050174265309547 and the zenith predicted is 1.5851644760505181
The azimuth is 5.029554633390068 and the zenith is 2.0874975005610428
The MAE is 2.545789300015317


2.545789300015317

In [ ]:
mae = find_error(False, batch1, sensors, directions)

200000
Event ID: 2621445
The azimuth predicted is -1.355191927815706 and the zenith predicted is 1.1412925667656684
The azimuth is 0.3273159930649267 and the zenith is 1.1463603360710157
The MAE is 1.4915900123680923
Event ID: 2097166
The azimuth predicted is -0.039685831473186224 and the zenith predicted is 1.1722266089448838
The azimuth is 5.982719680357331 and the zenith is 0.8897709814010096
The MAE is 0.3592396969462404
Event ID: 1048595
The azimuth predicted is 2.065952297769025 and the zenith predicted is 1.2230663629030336
The azimuth is 2.1833224189766867 and the zenith is 1.2382788137390635
The MAE is 0.11167878208776189
Event ID: 524309
The azimuth predicted is -1.5707963267948966 and the zenith predicted is 0.0
The azimuth is 2.0643916454211624 and the zenith is 0.8286687697666585
The MAE is 0.8286687697666585
Event ID: 3145750
The azimuth predicted is 1.9562631358087708 and the zenith predicted is 0.40719302801214846
The azimuth is 3.88099165476607 and the zenith is 1.7912

The azimuth is 3.659240668553121 and the zenith is 1.1474280031898736
The MAE is 0.6864554180993422
Event ID: 77
The azimuth predicted is -1.3258176636680326 and the zenith predicted is 0.0
The azimuth is 0.011371512389135186 and the zenith is 1.295370753782866
The MAE is 1.295370753782866
Event ID: 524365
The azimuth predicted is 0.1587450849417981 and the zenith predicted is 0.748358830711186
The azimuth is 3.5933756873894227 and the zenith is 0.7142687974142561
The MAE is 1.4434937166265074
Event ID: 79
The azimuth predicted is -2.76968767913042 and the zenith predicted is 2.417563754077166
The azimuth is 3.5333968141714607 and the zenith is 2.479946556392892
The MAE is 0.06366220356538785
Event ID: 82
The azimuth predicted is -0.9660006376648549 and the zenith predicted is 1.9841934246544857
The azimuth is 5.252107619089514 and the zenith is 0.6723655323859931
The MAE is 1.3130766756410601
Event ID: 2097242
The azimuth predicted is -2.983696464193952 and the zenith predicted is 1.2

The azimuth is 5.049452904244347 and the zenith is 0.4405094185883002
The MAE is 0.3257126085358426
Event ID: 2097324
The azimuth predicted is -1.4899208178662866 and the zenith predicted is 1.8825529119233868
The azimuth is 2.302391929831176 and the zenith is 0.38415996878107594
The MAE is 2.1751054738837987
Event ID: 1573037
The azimuth predicted is 0.7853981633974483 and the zenith predicted is 0.0
The azimuth is 5.739525972397524 and the zenith is 1.2365816752647254
The MAE is 1.2365816752647254
Event ID: 2621613
The azimuth predicted is 2.251106472188531 and the zenith predicted is 0.4710509672913548
The azimuth is 0.48668187938520013 and the zenith is 1.6515338942792102
The MAE is 1.7303797107098544
Event ID: 1573043
The azimuth predicted is -1.745862650694051 and the zenith predicted is 0.8492077769426656
The azimuth is 5.8207964186663945 and the zenith is 1.0017705892407849
The MAE is 1.0061033081177124
Event ID: 2097333
The azimuth predicted is -2.2481835314190683 and the zeni

The azimuth is 5.562590438482333 and the zenith is 1.596010832835314
The MAE is 2.064323635187474
Event ID: 3146006
The azimuth predicted is -0.8887591833582875 and the zenith predicted is 0.6693884182651373
The azimuth is 1.6190140042832766 and the zenith is 0.017382973492953733
The MAE is 0.6834606834447217
Event ID: 2097433
The azimuth predicted is -2.981016490528736 and the zenith predicted is 0.5825687384638039
The azimuth is 1.8288687323178934 and the zenith is 2.4950120254489
The MAE is 2.2578098720706743
Event ID: 290
The azimuth predicted is -1.9363746593260351 and the zenith predicted is 2.035524631607392
The azimuth is 4.6583209559830205 and the zenith is 1.625890423140408
The MAE is 0.5067882214034791
Event ID: 1048866
The azimuth predicted is -2.985512555618301 and the zenith predicted is 1.393482788163053
The azimuth is 0.4331793474299901 and the zenith is 0.259767993619282
The MAE is 1.6435769550589459
Event ID: 2097444
The azimuth predicted is 0.9017441781091516 and the

The azimuth is 2.578650263522506 and the zenith is 1.7581325980351632
The MAE is 1.7581325980351632
Event ID: 524665
The azimuth predicted is 2.25895332344063 and the zenith predicted is 0.4718140187674005
The azimuth is 1.1678720504443436 and the zenith is 1.2208126493799942
The MAE is 1.0443434253214012
Event ID: 1573237
The azimuth predicted is -2.7643921003367726 and the zenith predicted is 1.8977513906693686
The azimuth is 3.9792433190908336 and the zenith is 2.6145475455823664
The MAE is 0.7893522949326834
Event ID: 2621817
The azimuth predicted is -1.780121717003303 and the zenith predicted is 1.6802852087882179
The azimuth is 4.092730962146695 and the zenith is 1.1939919010641946
The MAE is 0.6311570184124148
Event ID: 524668
The azimuth predicted is -1.2379853392644622 and the zenith predicted is 0.9256598151681903
The azimuth is 5.089088388912805 and the zenith is 0.4919260492542664
The MAE is 0.4345976835378379
Event ID: 1573244
The azimuth predicted is -0.27497433867607285 

...

The azimuth is 1.234377821910959 and the zenith is 1.2662160606708264
The MAE is 1.207449801588003
Event ID: 2623435
The azimuth predicted is -1.454182801130948 and the zenith predicted is 2.6673075858438735
The azimuth is 2.8263349310288053 and the zenith is 2.090724261285626
The MAE is 1.2910859058546111
Event ID: 1050581
The azimuth predicted is -0.0 and the zenith predicted is 0.0
The azimuth is 0.04626179844102414 and the zenith is 1.237498398953253
The MAE is 1.237498398953253
Event ID: 1050583
The azimuth predicted is -2.9418251539229145 and the zenith predicted is 0.3768731557661346
The azimuth is 4.219128683164839 and the zenith is 0.43361446533171205
The MAE is 0.3406082478101935
Event ID: 2011
The azimuth predicted is 0.524017343001476 and the zenith predicted is 2.389360910703492
The azimuth is 0.3205339472740684 and the zenith is 2.314840905694404
The MAE is 0.1623315758818758
Event ID: 2623451
The azimuth predicted is 2.4817396959742557 and the zenith predicted is 0.32369

The azimuth is 2.442635810591735 and the zenith is 2.1384810977029893
The MAE is 1.6965496027801454
Event ID: 2099275
The azimuth predicted is 1.3247383919511533 and the zenith predicted is 0.522789967275193
The azimuth is 3.866479902407621 and the zenith is 0.9983002507026677
The MAE is 1.4475296900061951
Event ID: 1574988
The azimuth predicted is 0.1558818382578425 and the zenith predicted is 2.0182687806408595
The azimuth is 5.43756609471658 and the zenith is 2.0210262387745845
The MAE is 0.8945974495568757
Event ID: 526413
The azimuth predicted is 0.1568477072375183 and the zenith predicted is 0.6466325641803811
The azimuth is 6.24081769865006 and the zenith is 0.5763205021526608
The MAE is 0.13399953113458718
Event ID: 3147849
The azimuth predicted is -3.141592653589793 and the zenith predicted is 0.0
The azimuth is 5.711752335148337 and the zenith is 0.9460696951409572
The MAE is 0.9460696951409572
Event ID: 2127
The azimuth predicted is 1.2490457723982544 and the zenith predicte

The azimuth is 1.0541283089495253 and the zenith is 1.3920848714620562
The MAE is 0.15283190054603787
Event ID: 2623664
The azimuth predicted is 2.7261785926761357 and the zenith predicted is 1.7599530164444412
The azimuth is 1.9502350304085048 and the zenith is 0.6827861800442997
The MAE is 1.2698315163092864
Event ID: 3147951
The azimuth predicted is -3.141592653589793 and the zenith predicted is 0.0
The azimuth is 2.2158261326582567 and the zenith is 1.7758706862457916
The MAE is 1.7758706862457916
Event ID: 2230
The azimuth predicted is 1.1184965870786163 and the zenith predicted is 0.5697440739777256
The azimuth is 1.0491539764000366 and the zenith is 0.5553213520620213
The MAE is 0.03968786739964313
Event ID: 2099382
The azimuth predicted is 1.4906496340678055 and the zenith predicted is 2.5370555538769963
The azimuth is 1.483336613529641 and the zenith is 2.530337539844239
The MAE is 0.007910537677279846
Event ID: 2623674
The azimuth predicted is -0.9573518707747983 and the zeni

The azimuth is 3.8913997171036434 and the zenith is 1.7537595265580062
The MAE is 0.62048319426831
Event ID: 2623751
The azimuth predicted is 0.16057505774386566 and the zenith predicted is 0.6290718272551177
The azimuth is 5.991791694723144 and the zenith is 1.6505874472359987
The MAE is 1.0892157031725884
Event ID: 1050892
The azimuth predicted is -0.8511243085357713 and the zenith predicted is 0.9441203237254564
The azimuth is 4.858856051748974 and the zenith is 2.463874669456137
The MAE is 1.6009557078433703
Event ID: 2623756
The azimuth predicted is -3.141592653589793 and the zenith predicted is 0.0
The azimuth is 4.949664665377943 and the zenith is 0.742773229017642
The MAE is 0.742773229017642
Event ID: 2623757
The azimuth predicted is 2.677945044588987 and the zenith predicted is 3.141592653589793
The azimuth is 5.220986804794111 and the zenith is 3.0094902326275266
The MAE is 0.13210242096226651
Event ID: 1050897
The azimuth predicted is 0.7853981633974483 and the zenith predi

The azimuth is 1.947293943328994 and the zenith is 0.14708869134096153
The MAE is 0.23756702432947682
Event ID: 1050972
The azimuth predicted is 2.2539617117931643 and the zenith predicted is 2.2041687474386746
The azimuth is 1.545841699037342 and the zenith is 0.7651469506987416
The MAE is 1.5736248434595124
Event ID: 1575258
The azimuth predicted is -0.0 and the zenith predicted is 3.141592653589793
The azimuth is 5.114883757153432 and the zenith is 1.7750910372589128
The MAE is 1.3665016163308803
Event ID: 526687
The azimuth predicted is 0.35399873889298633 and the zenith predicted is 0.3517417765062015
The azimuth is 1.2819111009713973 and the zenith is 1.764431264311101
The MAE is 1.5487503869076642
Event ID: 3148128
The azimuth predicted is 2.2521852597535106 and the zenith predicted is 1.4831201232648488
The azimuth is 5.438532821704639 and the zenith is 1.5282205342728465
The MAE is 3.003880162767544
Event ID: 2623841
The azimuth predicted is -0.8888067909029298 and the zenith 

In [ ]:
mae.head()